In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG Engine Management

https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api#python_1



### Install Vertex AI SDK for Python

In [1]:
# @title Install packages
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,langchain,ag2] \
                                      google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.3/674.3 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [19]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai
%pip install --upgrade --user --quiet langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL_NAME = "gemini-2.0-flash-001"

In [4]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

!gcloud config set project ai-hangsik

Updated property [core/project].


### Package import and initialize model

In [5]:
# @title Initialize Vertex AI

import os

import vertexai
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool

from google import genai


vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## RAG Engine management

### Corpus managment helper functions

In [6]:
# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

#---------------------------------------------
def get_list_corpora():
  """
  Get corpora list.
  """
  corpora = rag.list_corpora()

  for corpus in corpora:
    print(f"[{corpus.create_time}][{corpus.display_name}][{corpus.name}]")

  return corpora

#---------------------------------------------

def get_corpus(corpus_name):
  """
  Get corpus.
  """
  corpus = rag.get_corpus(name=corpus_name)
  print(corpus)
  return corpus

#---------------------------------------------

def del_corpus(corpus_name):
  """
  Delete corpus.
  """

  rag.delete_corpus(name=corpus_name)
  print(f"Corpus {corpus_name} deleted. takes a little time to check the corpus is deleted.")


#### Get list of corpora that are registered

In [7]:
corpora = get_list_corpora()

[2025-03-04 08:34:38.256646+00:00][my-rag-corpus][projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760]
[2025-03-05 01:03:03.493126+00:00][my-rag-corpus][projects/ai-hangsik/locations/us-central1/ragCorpora/5359846506524311552]
[2025-03-05 01:20:47.154372+00:00][my-rag-corpus][projects/ai-hangsik/locations/us-central1/ragCorpora/1901081992703770624]
[2025-03-19 04:23:12.359628+00:00][my-rag-corpus][projects/ai-hangsik/locations/us-central1/ragCorpora/3379951520341557248]
[2025-03-19 05:02:42.994623+00:00][my-rag-corpus][projects/ai-hangsik/locations/us-central1/ragCorpora/6838716034162098176]


#### Get a corpus

In [10]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
get_corpus(corpus_name)

RagCorpus(name='projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760', display_name='my-rag-corpus', description='', embedding_model_config=EmbeddingModelConfig(publisher_model='projects/721521243942/locations/us-central1/publishers/google/models/text-embedding-005', endpoint=None, model=None, model_version_id=None), vector_db=RagManagedDb(), vertex_ai_search_config=None, backend_config=RagVectorDbConfig(vector_db=RagManagedDb(), rag_embedding_model_config=RagEmbeddingModelConfig(vertex_prediction_endpoint=VertexPredictionEndpoint(endpoint=None, publisher_model='projects/721521243942/locations/us-central1/publishers/google/models/text-embedding-005', model=None, model_version_id=None))))


RagCorpus(name='projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760', display_name='my-rag-corpus', description='', embedding_model_config=EmbeddingModelConfig(publisher_model='projects/721521243942/locations/us-central1/publishers/google/models/text-embedding-005', endpoint=None, model=None, model_version_id=None), vector_db=RagManagedDb(), vertex_ai_search_config=None, backend_config=RagVectorDbConfig(vector_db=RagManagedDb(), rag_embedding_model_config=RagEmbeddingModelConfig(vertex_prediction_endpoint=VertexPredictionEndpoint(endpoint=None, publisher_model='projects/721521243942/locations/us-central1/publishers/google/models/text-embedding-005', model=None, model_version_id=None))))

#### delete a corpus

In [11]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
# del_corpus(corpus_name)

### File management helper functions

In [12]:
# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

#---------------------------------------------
def upload_file(corpus_name,
                path,
                display_name,
                description
                ):
  """
  Upload file
  """

  rag_file = rag.upload_file(
    corpus_name=corpus_name,
    path=path,
    display_name=display_name,
    description=description,
  )

  print(rag_file)

  return rag_file

#---------------------------------------------

def import_files(corpus_name,
                 paths,
                 chunk_size,
                 chunk_overlap
                 ):
  """
  Get corpus.

  # corpus_name = "projects/{PROJECT_ID}/locations/us-central1/ragCorpora/{rag_corpus_id}"
  # paths = ["https://drive.google.com/file/123", "gs://my_bucket/my_files_dir"]  # Supports Google Cloud Storage and Google Drive Links

  """

  response = rag.import_files(
      corpus_name=corpus_name,
      paths=paths,
      chunk_size=chunk_size,  # Optional
      chunk_overlap=chunk_overlap,  # Optional
      max_embedding_requests_per_min=900,  # Optional
  )

  print(f"Imported {response.imported_rag_files_count} files.")

  return response

#---------------------------------------------

def get_rag_files(corpus_name):
  """
  Get files
  """

  files = rag.list_files(corpus_name=corpus_name)
  for file in files:
      print(file.display_name)
      print(file.name)

  return files

#---------------------------------------------

def get_rag_file(file_name):
  """
  Get a file.
  """
  rag_file = rag.get_file(name=file_name)
  print(rag_file)
  return rag_file

#---------------------------------------------

def del_lag_file(file_name):
  """
  Delete file.
  """

  rag.delete_file(name=file_name)
  print(f"File {file_name} deleted. takes a little time to check the file is deleted.")


#### upload rag file from local stroage

In [13]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
path = "./google-ai-agents-whitepaper.pdf"
display_name = "ai-agents-whitepaper"
description = "ai-agents-whitepaper desc"

rag_file = upload_file(corpus_name,
                path,
                display_name,
                description
                )
print(rag_file)

RagFile(name='projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603022199990948', display_name='ai-agents-whitepaper', description='ai-agents-whitepaper desc')
RagFile(name='projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603022199990948', display_name='ai-agents-whitepaper', description='ai-agents-whitepaper desc')


#### import rag files from Drive or GCS

In [14]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
paths = ["gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf",
        "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf",
        ]

rag_files = import_files(corpus_name,
                paths,
                chunk_size=512,
                chunk_overlap=100

                )

print(rag_files)

Imported 2 files.
imported_rag_files_count: 2



#### get rag file list

In [15]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
get_rag_files(corpus_name)

개인정보 보호법(법률)(제19234호)(20240315).pdf
projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5381842860533614422
신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf
projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5381842872291213234
정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf
projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5381842884034109271
ai-agents-whitepaper
projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603022199990948
DaouOffice 클라우드 관리자 가이드.pdf
projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603430846561011
DaouOffice 클라우드 서비스 가이드 3.5.21.pdf
projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603973484696266


ListRagFilesPager<rag_files {
  gcs_source {
    uris: "gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf"
  }
  name: "projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5381842860533614422"
  display_name: "개인정보 보호법(법률)(제19234호)(20240315).pdf"
  create_time {
    seconds: 1741077292
    nanos: 197356000
  }
  update_time {
    seconds: 1741077292
    nanos: 197356000
  }
  file_status {
    state: ACTIVE
  }
}
rag_files {
  gcs_source {
    uris: "gs://it_laws_kr/law_pdf/신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
  }
  name: "projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5381842872291213234"
  display_name: "신용정보의 이용 및 보호에 관한 법률(법률)(제19234호)(20230915).pdf"
  create_time {
    seconds: 1741077293
    nanos: 766700000
  }
  update_time {
    seconds: 1741077293
    nanos: 766700000
  }
  file_status {
    stat

#### get rag file

In [13]:
file_name = "projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603973484696266"
get_rag_file(file_name)

RagFile(name='projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603973484696266', display_name='DaouOffice 클라우드 서비스 가이드 3.5.21.pdf', description='')


RagFile(name='projects/721521243942/locations/us-central1/ragCorpora/2662753279682805760/ragFiles/5392603973484696266', display_name='DaouOffice 클라우드 서비스 가이드 3.5.21.pdf', description='')

#### delete a rag file

In [14]:
file_name = "projects/721521243942/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5309373647786126728"

# del_lag_file(file_name)

### Retrival query

In [15]:
def retrieve(corpus_name, query):

  response = rag.retrieval_query(
      rag_resources=[
          rag.RagResource(
              rag_corpus=corpus_name,
              # Optional: supply IDs from `rag.list_files()`.
              # rag_file_ids=["rag-file-1", "rag-file-2", ...],
          )
      ],
      text=query,
      similarity_top_k=10,  # Optional
      vector_distance_threshold=0.5,  # Optional
  )
  print(response)

def generate_answer(corpus_name, query):

  rag_retrieval_tool = Tool.from_retrieval(
      retrieval=rag.Retrieval(
          source=rag.VertexRagStore(
              rag_resources=[
                  rag.RagResource(
                      rag_corpus=corpus_name,
                      # Optional: supply IDs from `rag.list_files()`.
                      # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                  )
              ],
              similarity_top_k=3,  # Optional
              vector_distance_threshold=0.5,  # Optional
          ),
      )
  )

  rag_model = GenerativeModel(
      model_name="gemini-1.5-flash-002", tools=[rag_retrieval_tool]
  )
  response = rag_model.generate_content(query)
  print(response.text)


In [16]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"
query = "대쉬보드란 무엇인가요?"

retrieve(corpus_name, query)


contexts {
  contexts {
    source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf"
    source_display_name: "DaouOffice 클라우드 서비스 가이드 3.5.21.pdf"
    text: ". . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 369| 17\r\n부서 인사정보\r\n전사 인사정보\r\n전사 인사정보 설정하기\r\n경리회계 경영지원을 사용하나요?  \r\n내 인사정보\r\n전사 인사정보\r\n16."
    distance: 0.35350941682996462
    score: 0.35350941682996462
  }
  contexts {
    source_uri: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf"
    source_display_name: "DaouOffice 클라우드 관리자 가이드.pdf"
    text: "1. 사이트 어드민의 [메뉴관리 > 보고 > 공용 양식]를 클릭합니다.보고 | 175\r\n2. 보고서 목록 상단에 있는 새 양식을 클릭합니다. 새 양식 추가화면이 나타납니다.\r\n3. 양식 제목을 사용자가 알아보기 쉽도록 1자 이상 100자 이하로 입력합니다.\r\n4. 양식 편집기에서 입력 요소를 선택합니다. 입력 요소를 선택하면, 해당 요소가 편집기에는 코드로 변환되어 삽\r\n입됩니다.\r\n예를 들어, 메모를 선택하면 편집기에는 {{textarea}}가 삽입되지만, 보고서 작성화면에서는 여러 줄을 입력할 수\r\n있는

## Integrate with Reasoning engine

### Helper methods

In [17]:
# Initialize session history
store = {}

def get_session_history(session_id: str):
  """
    Get session history
  """
  from langchain.memory import ChatMessageHistory

  if session_id not in store:
      store[session_id] = ChatMessageHistory()
  return store[session_id]

#---------------------------------------------------------------
def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']

#---------------------------------------------------------------

def create_agent(rag_corpus_name):
  from vertexai.preview.reasoning_engines import LangchainAgent

  from googleapiclient import discovery
  from langchain.agents.format_scratchpad.tools import format_to_tool_messages
  from langchain_core import prompts

  # Define prompt template
  prompt = {
      "history": lambda x: x["history"],
      "input": lambda x: x["input"],
      "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
  } | prompts.ChatPromptTemplate.from_messages(
      [
          prompts.MessagesPlaceholder(variable_name="history"),
          ("user", "{input}"),
          prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
      ]
  )

  rag_retrieval_tool = Tool.from_retrieval(
      retrieval=rag.Retrieval(
          source=rag.VertexRagStore(
              rag_resources=[
                  rag.RagResource(
                      rag_corpus=rag_corpus_name,  # Currently only 1 corpus is allowed.
                      # Optional: supply IDs from `rag.list_files()`.
                      # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                  )
              ],
              similarity_top_k=10,  # Optional
              vector_distance_threshold=0.5,  # Optional
          ),
      )
  )

  agent = LangchainAgent(
      prompt=prompt,
      model=MODEL_NAME,
      chat_history=get_session_history,
      model_kwargs={"temperature": 0},
      tools=[rag_retrieval_tool],
      agent_executor_kwargs={"return_intermediate_steps": True},
  )

  return agent


In [ ]:
from time import perf_counter
import langchain_google_vertexai

session_id = "sess_101"
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/2662753279682805760"

agent = create_agent(corpus_name)

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)